In [4]:
# INSURANCE COST PREDICTION - COMPLETE MACHINE LEARNING PROJECT

# =============================================================================
# 1. LIBRARIES IMPORTATION
# =============================================================================

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# =============================================================================
# 2. DATASET LOADING AND INITIAL EXPLORATION
# =============================================================================

df = pd.read_csv('insurance.csv', sep=',')

print("=" * 80)
print("DATASET OVERVIEW")
print("=" * 80)
print(f"\nDataset shape: {df.shape[0]} rows, {df.shape[1]} columns\n")

print("\nFirst rows:")
print(df.head())

print("\n\nDataset information:")
print(df.info())

print("\n\nVariables definition:")
print("""
- age: Age of the policyholder (integer)
- sex: Gender of the policyholder (categorical: male/female)
- bmi: Body Mass Index (float)
- children: Number of children/dependents (integer)
- smoker: Smoking status (categorical: yes/no)
- region: Residential area (categorical: northeast/northwest/southeast/southwest)
- charges: Individual medical costs billed by insurance (float) [TARGET VARIABLE]
""")

# =============================================================================
# 3. DATA QUALITY ANALYSIS
# =============================================================================

print("=" * 80)
print("DATA QUALITY ANALYSIS")
print("=" * 80)

print("\nMissing values per column:")
print(df.isnull().sum())

print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Remove duplicates if any
if df.duplicated().sum() > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")

# =============================================================================
# 4. DESCRIPTIVE STATISTICS
# =============================================================================

print("\n" + "=" * 80)
print("DESCRIPTIVE STATISTICS")
print("=" * 80)

print("\nNumerical variables summary:")
print(df.describe())

print("\n\nCategorical variables distribution:")
for col in ['sex', 'smoker', 'region']:
    print(f"\n{col.upper()}:")
    print(df[col].value_counts())
    print(f"Proportion:\n{df[col].value_counts(normalize=True) * 100}")

# =============================================================================
# 5. DATA VISUALIZATION
# =============================================================================

print("\n" + "=" * 80)
print("GENERATING VISUALIZATIONS")
print("=" * 80)

# Distribution of numerical variables
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
numerical_cols = ['age', 'bmi', 'children', 'charges']

for i, col in enumerate(numerical_cols):
    ax = axes[i//3, i%3]
    df[col].hist(bins=30, edgecolor='black', ax=ax, color='skyblue')
    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('distributions.png', dpi=300, bbox_inches='tight')
plt.show()

# Categorical variables visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.countplot(data=df, x='smoker', palette='Set2', ax=axes[0])
axes[0].set_title('Smoker Distribution')

sns.countplot(data=df, x='sex', palette='pastel', ax=axes[1])
axes[1].set_title('Gender Distribution')

sns.countplot(data=df, x='region', palette='viridis', ax=axes[2])
axes[2].set_title('Region Distribution')

plt.tight_layout()
plt.savefig('categorical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

# Charges analysis by categorical variables
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

sns.boxplot(data=df, x='smoker', y='charges', palette='coolwarm', ax=axes[0])
axes[0].set_title('Charges by Smoker Status')

sns.boxplot(data=df, x='sex', y='charges', palette='Set1', ax=axes[1])
axes[1].set_title('Charges by Gender')

sns.boxplot(data=df, x='region', y='charges', palette='Set3', ax=axes[2])
axes[2].set_title('Charges by Region')

plt.tight_layout()
plt.savefig('charges_by_category.png', dpi=300, bbox_inches='tight')
plt.show()

# Correlation analysis
plt.figure(figsize=(10, 8))
df_encoded = df.copy()
df_encoded['sex'] = df_encoded['sex'].map({'female': 0, 'male': 1})
df_encoded['smoker'] = df_encoded['smoker'].map({'no': 0, 'yes': 1})
correlation_matrix = df_encoded[['age', 'sex', 'bmi', 'children', 'smoker', 'charges']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True, linewidths=1)
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nKey correlations with charges:")
print(correlation_matrix['charges'].sort_values(ascending=False))

# Scatter plots for important relationships
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

sns.scatterplot(data=df, x='age', y='charges', hue='smoker', palette='Set1', ax=axes[0,0], alpha=0.6)
axes[0,0].set_title('Age vs Charges (colored by Smoker)')

sns.scatterplot(data=df, x='bmi', y='charges', hue='smoker', palette='Set1', ax=axes[0,1], alpha=0.6)
axes[0,1].set_title('BMI vs Charges (colored by Smoker)')

sns.scatterplot(data=df, x='age', y='bmi', hue='smoker', palette='Set1', ax=axes[1,0], alpha=0.6)
axes[1,0].set_title('Age vs BMI (colored by Smoker)')

sns.violinplot(data=df, x='children', y='charges', palette='muted', ax=axes[1,1])
axes[1,1].set_title('Charges by Number of Children')

plt.tight_layout()
plt.savefig('scatter_plots.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# 6. OUTLIERS DETECTION AND TREATMENT
# =============================================================================

print("\n" + "=" * 80)
print("OUTLIERS ANALYSIS")
print("=" * 80)

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check outliers for numerical columns
numerical_cols = ['age', 'bmi', 'children', 'charges']
outliers_summary = {}

for col in numerical_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outliers_summary[col] = len(outliers)
    print(f"\n{col.upper()}:")
    print(f"  Outliers detected: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
    print(f"  Range: [{lower:.2f}, {upper:.2f}]")

# Visualize outliers
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, col in enumerate(numerical_cols):
    axes[i].boxplot(df[col], vert=True)
    axes[i].set_title(f'{col} - Outliers')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.savefig('outliers_boxplots.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nDecision: Keep outliers as they represent valid high-cost insurance cases")



# 7. DATA PREPROCESSING
print("\n" + "=" * 80)
print("DATA PREPROCESSING")
print("=" * 80)

# Encodage des variables catégorielles
df['sex'] = df['sex'].map({'female': 0, 'male': 1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes': 1})
df = pd.get_dummies(df, columns=['region'], drop_first=True)

# Séparation des variables explicatives et de la cible
X = df.drop('charges', axis=1)
y = df['charges']

# Mise à l’échelle des variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Découpage en train et test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)
print("Preprocessing completed successfully")

# BASELINE : Linear Regression
print("\n" + "=" * 80)
print("BASELINE MODEL - Linear Regression")
print("=" * 80)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred_lin = lin_reg.predict(X_test)

print("\nLinear Regression:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_lin), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_lin)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_lin), 4))

# REGULARIZED MODELS : Ridge & Lasso
print("\n" + "=" * 80)
print("REGULARIZED LINEAR MODELS")
print("=" * 80)

# Ridge Regression
ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

print("\nRidge Regression:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_ridge), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_ridge)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_ridge), 4))

# Lasso Regression
lasso = Lasso(alpha=1.0, random_state=42)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

print("\nLasso Regression:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_lasso), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_lasso)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_lasso), 4))

# TREE-BASED MODELS
print("\n" + "=" * 80)
print("TREE-BASED MODELS")
print("=" * 80)

# Decision Tree
dt = DecisionTreeRegressor(max_depth=10, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

print("\nDecision Tree:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_dt), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_dt)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_dt), 4))

# Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("\nRandom Forest:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_rf), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_rf)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_rf), 4))


# TREE-BASED MODELS
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve


# BASELINE : Linear Regression
print("\n" + "=" * 80)
print("BASELINE MODEL - Linear Regression")
print("=" * 80)

lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)
y_pred_lin = lin_reg.predict(X_test_scaled)

print("\nLinear Regression:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_lin), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_lin)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_lin), 4))


# REGULARIZED MODEL : Ridge
print("\n" + "=" * 80)
print("REGULARIZED MODEL - Ridge Regression")
print("=" * 80)

ridge = Ridge(alpha=1.0, random_state=42)
ridge.fit(X_train_scaled, y_train)
y_pred_ridge = ridge.predict(X_test_scaled)

print("\nRidge Regression:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_ridge), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_ridge)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_ridge), 4))


# TREE-BASED MODEL : Decision Tree
print("\n" + "=" * 80)
print("TREE-BASED MODEL - Decision Tree")
print("=" * 80)

dt = DecisionTreeRegressor(max_depth=10, random_state=42)
dt.fit(X_train_scaled, y_train)
y_pred_dt = dt.predict(X_test_scaled)

print("\nDecision Tree:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_dt), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_dt)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_dt), 4))



# ENSEMBLE MODEL : Random Forest
print("\n" + "=" * 80)
print("ENSEMBLE MODEL - Random Forest")
print("=" * 80)

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

print("\nRandom Forest:")
print("  MAE :", round(mean_absolute_error(y_test, y_pred_rf), 2))
print("  RMSE:", round(np.sqrt(mean_squared_error(y_test, y_pred_rf)), 2))
print("  R²  :", round(r2_score(y_test, y_pred_rf), 4))


# LEARNING CURVES ANALYSIS
print("\n" + "=" * 80)
print("LEARNING CURVES ANALYSIS")
print("=" * 80)

# Courbes d'apprentissage = évolution des performances quand on augmente la taille du jeu d'entraînement
train_sizes, train_scores, val_scores = learning_curve(
    lin_reg, X_train_scaled, y_train,
    cv=5, scoring='neg_mean_absolute_error',
    train_sizes=np.linspace(0.1, 1.0, 5), n_jobs=-1
)

# Moyenne des scores
train_mean = -train_scores.mean(axis=1)
val_mean = -val_scores.mean(axis=1)

# Tracé des courbes
plt.plot(train_sizes, train_mean, label='Training MAE', color='blue', marker='o')
plt.plot(train_sizes, val_mean, label='Validation MAE', color='red', marker='o')
plt.xlabel('Training Set Size')
plt.ylabel('Mean Absolute Error')
plt.title('Learning Curve - Linear Regression')
plt.legend()
plt.show()

# Résultats finaux
print("\nLinear Regression:")
print("  Training MAE :", round(train_mean[-1], 2))
print("  Validation MAE :", round(val_mean[-1], 2))
print("  Gap :", round(val_mean[-1] - train_mean[-1], 2))




# =============================================================================
# 13. FEATURE IMPORTANCE ANALYSIS
# =============================================================================

print("\n" + "=" * 80)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 80)

# Random Forest feature importance
feature_importance_rf = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nRANDOM FOREST - Feature Importance:")
print(feature_importance_rf)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_rf, x='Importance', y='Feature', palette='viridis')
plt.title('Feature Importance - Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_importance_rf.png', dpi=300, bbox_inches='tight')
plt.show()

# Gradient Boosting feature importance
feature_importance_gb = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_gb.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nGRADIENT BOOSTING - Feature Importance:")
print(feature_importance_gb)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_gb, x='Importance', y='Feature', palette='plasma')
plt.title('Feature Importance - Gradient Boosting')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('feature_importance_gb.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# 14. DIMENSIONALITY REDUCTION - PCA
# =============================================================================

print("\n" + "=" * 80)
print("DIMENSIONALITY REDUCTION - PCA ANALYSIS")
print("=" * 80)

# Apply PCA
pca = PCA()
X_train_pca = pca.fit_transform(X_train_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

print("\nExplained variance by component:")
for i, var in enumerate(explained_variance):
    print(f"  PC{i+1}: {var:.4f} ({cumulative_variance[i]:.4f} cumulative)")

# Plot explained variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='steelblue')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('PCA - Explained Variance by Component')
axes[0].grid(True, alpha=0.3)

axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-', color='darkred')
axes[1].axhline(y=0.95, color='green', linestyle='--', label='95% threshold')
axes[1].set_xlabel('Number of Components')
axes[1].set_ylabel('Cumulative Explained Variance')
axes[1].set_title('PCA - Cumulative Explained Variance')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('pca_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Find number of components for 95% variance
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nNumber of components to explain 95% variance: {n_components_95}")

# Train model with reduced dimensions
if n_components_95 < X_train_scaled.shape[1]:
    pca_reduced = PCA(n_components=n_components_95)
    X_train_pca_reduced = pca_reduced.fit_transform(X_train_scaled)
    X_test_pca_reduced = pca_reduced.transform(X_test_scaled)
    
    rf_pca = RandomForestRegressor(**grid_rf.best_params_, random_state=42)
    rf_pca.fit(X_train_pca_reduced, y_train)
    y_pred_rf_pca = rf_pca.predict(X_test_pca_reduced)
    
    mae_rf_pca = mean_absolute_error(y_test, y_pred_rf_pca)
    r2_rf_pca = r2_score(y_test, y_pred_rf_pca)
    
    print(f"\nRandom Forest with PCA ({n_components_95} components):")
    print(f"  MAE: {mae_rf_pca:.2f}")
    print(f"  R²: {r2_rf_pca:.4f}")
    print(f"  Performance change: {((mae_rf - mae_rf_pca) / mae_rf * 100):.2f}%")

# =============================================================================
# 15. RESIDUALS ANALYSIS
# =============================================================================

print("\n" + "=" * 80)
print("RESIDUALS ANALYSIS")
print("=" * 80)

# Calculate residuals for best model (Gradient Boosting)
residuals = y_test - y_pred_gb

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Residuals vs Predicted
axes[0, 0].scatter(y_pred_gb, residuals, alpha=0.6, edgecolors='k')
axes[0, 0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].set_xlabel('Predicted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Predicted Values')
axes[0, 0].grid(True, alpha=0.3)

# Histogram of residuals
axes[0, 1].hist(residuals, bins=30, edgecolor='black', alpha=0.7, color='skyblue')
axes[0, 1].set_xlabel('Residuals')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Residuals')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot')
axes[1, 0].grid(True, alpha=0.3)

# Actual vs Predicted
axes[1, 1].scatter(y_test, y_pred_gb, alpha=0.6, edgecolors='k')
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', linewidth=2)
axes[1, 1].set_xlabel('Actual Values')
axes[1, 1].set_ylabel('Predicted Values')
axes[1, 1].set_title('Actual vs Predicted Values')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# Residuals statistics
print("\nResiduals Statistics:")
print(f"  Mean: {residuals.mean():.2f}")
print(f"  Std Dev: {residuals.std():.2f}")
print(f"  Min: {residuals.min():.2f}")
print(f"  Max: {residuals.max():.2f}")

# =============================================================================
# 16. FINAL MODEL COMPARISON
# =============================================================================

print("\n" + "=" * 80)
print("FINAL MODEL COMPARISON")
print("=" * 80)

results = pd.DataFrame({
    'Model': [
        'Linear Regression',
        'Ridge Regression',
        'Lasso Regression',
        'ElasticNet',
        'Decision Tree',
        'Random Forest (tuned)',
        'Gradient Boosting (tuned)',
        'Voting Ensemble',
        'Stacking Ensemble'
    ],
    'MAE': [mae_test_lr, mae_ridge, mae_lasso, mae_elasticnet, mae_dt, 
            mae_rf, mae_gb, mae_voting, mae_stacking],
    'RMSE': [rmse_test_lr, rmse_ridge, rmse_lasso, rmse_elasticnet, rmse_dt,
             rmse_rf, rmse_gb, rmse_voting, rmse_stacking],
    'R²': [r2_test_lr, r2_ridge, r2_lasso, r2_elasticnet, r2_dt,
           r2_rf, r2_gb, r2_voting, r2_stacking]
})

results = results.sort_values(by='MAE', ascending=True)
print("\n", results.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.barplot(data=results, x='MAE', y='Model', palette='viridis', ax=axes[0])
axes[0].set_title('Mean Absolute Error Comparison')
axes[0].set_xlabel('MAE')

sns.barplot(data=results, x='RMSE', y='Model', palette='plasma', ax=axes[1])
axes[1].set_title('Root Mean Squared Error Comparison')
axes[1].set_xlabel('RMSE')

sns.barplot(data=results, x='R²', y='Model', palette='coolwarm', ax=axes[2])
axes[2].set_title('R² Score Comparison')
axes[2].set_xlabel('R² Score')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# =============================================================================
# 17. BEST MODEL SELECTION AND FINAL EVALUATION
# =============================================================================

print("\n" + "=" * 80)
print("BEST MODEL SELECTION")
print("=" * 80)

best_model_name = results.iloc[0]['Model']
best_mae = results.iloc[0]['MAE']
best_rmse = results.iloc[0]['RMSE']
best_r2 = results.iloc[0]['R²']

print(f"\nBest performing model: {best_model_name}")
print(f"  MAE: {best_mae:.2f}")
print(f"  RMSE: {best_rmse:.2f}")
print(f"  R²: {best_r2:.4f}")

# Calculate MAPE for best model
if best_model_name == 'Gradient Boosting (tuned)':
    best_model = best_gb
    y_pred_best = y_pred_gb
elif best_model_name == 'Random Forest (tuned)':
    best_model = best_rf
    y_pred_best = y_pred_rf
elif best_model_name == 'Stacking Ensemble':
    best_model = stacking
    y_pred_best = y_pred_stacking
else:
    best_model = voting
    y_pred_best = y_pred_voting

mape = mean_absolute_percentage_error(y_test, y_pred_best)
print(f"  MAPE: {mape*100:.2f}%")

# =============================================================================
# 18. CONCLUSIONS AND RECOMMENDATIONS
# =============================================================================

print("\n" + "=" * 80)
print("CONCLUSIONS AND RECOMMENDATIONS")
print("=" * 80)

print("""
KEY FINDINGS:

1. DATA INSIGHTS:
   - Smoking status is the strongest predictor of insurance charges
   - Strong correlation between age and charges, especially for smokers
   - BMI shows moderate correlation with charges
   - Gender and region have minimal impact on charges

2. MODEL PERFORMANCE:
   - Tree-based ensemble methods (RF, GB) significantly outperform linear models
   - Gradient Boosting achieved the best balance of accuracy and generalization
   - Ensemble methods (Voting/Stacking) provide stable predictions
   - Linear models show underfitting, while complex trees show slight overfitting

3. FEATURE IMPORTANCE:
   - Top predictors: smoker status, age, bmi
   - Number of children and region have lower importance
   - PCA analysis shows most variance captured by few components

4. RECOMMENDATIONS:
   - Use Gradient Boosting or ensemble models for production
   - Focus data collection on key features: smoking status, age, BMI
   - Consider interaction features (age × smoker, BMI × smoker)
   - Monitor model performance regularly and retrain with new data

REFERENCES:
- Breiman, L. (2001). Random Forests. Machine Learning, 45(1), 5-32.
- Friedman, J. H. (2001). Greedy function approximation: A gradient boosting machine.
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). The Elements of Statistical Learning.
- Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System.

PROJECT COMPLETED SUCCESSFULLY
""")

DATASET OVERVIEW

Dataset shape: 1338 rows, 1 columns


First rows:
  age;sex;bmi;children;smoker;region;charges
0   19;female;27.9;0;yes;southwest;16884.924
1     18;male;33.77;1;no;southeast;1725.5523
2         28;male;33;3;no;southeast;4449.462
3  33;male;22.705;0;no;northwest;21984.47061
4     32;male;28.88;0;no;northwest;3866.8552


Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 1 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   age;sex;bmi;children;smoker;region;charges  1338 non-null   object
dtypes: object(1)
memory usage: 10.6+ KB
None


Variables definition:

- age: Age of the policyholder (integer)
- sex: Gender of the policyholder (categorical: male/female)
- bmi: Body Mass Index (float)
- children: Number of children/dependents (integer)
- smoker: Smoking status (categorical: yes/no)
- region: Resi

KeyError: 'sex'